**d)** First we import our work horses and load a `DataFrame`

In [2]:
import numpy as np
import pandas as pd
import scipy.optimize
import numba
import matplotlib.pyplot as plt
import seaborn as sns

# Magic function to make matplotlib inline
%matplotlib inline

# import Justin's settings
rc = {'lines.linewidth': 2,
      'axes.labelsize': 18,
      'axes.titlesize': 18,
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

df = pd.read_csv('./data/mean_rest_bouts.csv', comment = '#')

In [3]:
# Since there are two fish whose rest bouts in a minute interveral
# are undetectable we will use .fillna to set them to zero

wt = df[df['genotype'] == 'wt'].fillna(0)

mut = df[df['genotype'] == 'mut'].fillna(0)

# we say that mu is our mean and sigma^2 is n / (n - 1) of our sample variance
# compute the mean and the variance for our wt and mutant data sets 

wt_mean = wt['mean_rest_bout_length'].mean()
wt_var = wt['mean_rest_bout_length'].var(ddof=1)

mut_mean = mut['mean_rest_bout_length'].mean()
mut_var = mut['mean_rest_bout_length'].var(ddof=1)

print ("""
           µ         σ          n
        ---------------------------
WT      {0:.4f}     {2:.4f}      {4:d}
Venus   {1:.4f}     {3:.4f}       {5:d}
""".format(wt_mean, mut_mean, np.sqrt(wt_var), np.sqrt(mut_var), 
           len(wt['mean_rest_bout_length']), len(mut['mean_rest_bout_length'])))


           µ         σ          n
        ---------------------------
WT      2.2094     0.5229      17
Venus   1.7271     0.8051       22



In [4]:
# we want to make an array of the mean rest bout lengths
wt = wt['mean_rest_bout_length'].values

mut = mut['mean_rest_bout_length'].values


In [5]:
# our null hypothesis is that the mutant and the wild type are drawn from the same
# Gaussian distribution in other words we want to test if the mutant mean is the same
# as the wt mean

# We first combine all of the measurements and then distrbute them to a set of 17 and 
# a set of 22

def distribute_samples(w, m, some_func, trials, args=()):
    '''
    combine our measurements for rest bout and redistribute them to two groups
    '''
    
    # first we create an empty array to store our samples
    samples = np.empty(trials)
    
    # concatenate all the measurements
    measurements = np.concatenate((w, m))
    
    # create sets for all trials 
    for i in range(trials):
        measurements = np.random.permutation(measurements)
        samples[i] = some_func(measurements[:len(w)], measurements[len(w):], *args)
     
    return samples


# now I want to define the p-value
def p_value(samples, actual, greater_or_less='greater'):
    '''
    compute the p value (the probability our measurement is 
    greather than or less than the actual value
    '''
    
    if greater_or_less == 'greater':
        return np.sum(samples >= actual) / len(samples)
    else:
        return np.sum(samples <= actual) / len(samples)

# now we can run our test and see if we will get the actual difference in our means
# this will be my some_func in my distribute_samples function
def abs_diff(w, m, return_abs=False):
    '''
    Calculate the absolute difference of the means
    '''
    diff = w.mean() - m.mean()
    if return_abs:
        return np.abs(diff)
    return diff

In [6]:
# let's retrieve all our random sample sets of two
samples = distribute_samples(wt, mut, abs_diff, 10000, args = (True,))

# calculate p value
p_value = p_value(samples, abs_diff(wt, mut, True))

print ('p value =', p_value)

p value = 0.0374
